In [6]:
import pandas as pd
import numpy as np
import akshare as ak
import warnings
from typing import List, Union, Optional, Dict
import time
from datetime import datetime, timedelta
import os
import json
from pathlib import Path
import concurrent.futures
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [23]:
import akshare as ak
import pandas as pd
import numpy as np
import warnings
from typing import List, Union, Optional, Dict
import time
from datetime import datetime, timedelta
import os
import json
from pathlib import Path
import concurrent.futures
from tqdm import tqdm  # 进度条显示

warnings.filterwarnings('ignore')

class ASshare:
    """
    A股数据获取类
    支持批量获取股票历史数据、基本面数据、因子数据等

    Attributes:
        cache_dir: 缓存目录路径
        max_workers: 最大并发线程数
        retry_times: 失败重试次数
    """

    def __init__(self, cache_dir: str = "./data_cache", max_workers: int = 5):
        """
        初始化ASshare类

        Args:
            cache_dir: 缓存目录，默认当前目录下的data_cache
            max_workers: 并发下载的最大线程数
        """
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.max_workers = max_workers
        self.retry_times = 3
        self.request_delay = 0.1  # 请求延迟，避免被封IP

        # 数据源配置
        self.data_sources = {
            'daily': ak.stock_zh_a_hist,
            'minute': ak.stock_zh_a_minute,
            'fundamental': ak.stock_financial_report_sina,
            # 可根据需要添加更多数据源
        }

    def _get_cache_key(self, symbol: str, data_type: str, **kwargs) -> str:
        """
        生成缓存键值

        Args:
            symbol: 股票代码
            data_type: 数据类型
            **kwargs: 其他参数

        Returns:
            缓存文件名
        """
        param_str = "_".join([f"{k}-{v}" for k, v in sorted(kwargs.items())])
        return f"{symbol}_{data_type}_{param_str}.parquet"

    def _save_to_cache(self, df: pd.DataFrame, cache_key: str):
        """
        保存数据到缓存

        Args:
            df: 数据框
            cache_key: 缓存键值
        """
        if df is not None and not df.empty:
            cache_path = self.cache_dir / cache_key
            try:
                df.to_parquet(cache_path)
            except Exception as e:
                # 如果parquet保存失败，使用csv作为备选
                cache_path = cache_path.with_suffix('.csv')
                df.to_csv(cache_path)

    def _load_from_cache(self, cache_key: str, max_cache_days: int = 1) -> Optional[pd.DataFrame]:
        """
        从缓存加载数据

        Args:
            cache_key: 缓存键值
            max_cache_days: 最大缓存天数，超过则重新下载

        Returns:
            数据框或None
        """
        # 尝试不同的文件格式
        for ext in ['.parquet', '.csv']:
            cache_path = self.cache_dir / cache_key.replace('.parquet', ext)
            if cache_path.exists():
                # 检查缓存是否过期
                cache_time = datetime.fromtimestamp(cache_path.stat().st_mtime)
                if (datetime.now() - cache_time).days < max_cache_days:
                    try:
                        if ext == '.parquet':
                            return pd.read_parquet(cache_path)
                        else:
                            return pd.read_csv(cache_path, index_col=0, parse_dates=True)
                    except Exception as e:
                        print(f"加载缓存失败 {cache_path}: {e}")
                        return None
        return None

    def _fetch_single_stock(self, symbol: str, **kwargs) -> Optional[pd.DataFrame]:
        """
        获取单只股票数据（带重试机制）

        Args:
            symbol: 股票代码
            **kwargs: 查询参数

        Returns:
            股票数据框或None
        """
        data_type = kwargs.pop('data_type', 'daily')

        # 检查缓存
        cache_key = self._get_cache_key(symbol, data_type, **kwargs)
        cached_data = self._load_from_cache(cache_key)
        if cached_data is not None:
            return cached_data

        # 从数据源获取
        if data_type not in self.data_sources:
            print(f"不支持的数据类型: {data_type}")
            return None

        func = self.data_sources[data_type]

        # 重试机制
        for attempt in range(self.retry_times):
            try:
                # 添加股票代码列以便后续识别
                df = func(symbol=symbol, **{k: v for k, v in kwargs.items()
                                          if k not in ['data_type']})

                if df is not None and not df.empty:
                    df['symbol'] = symbol

                    # 标准化列名
                    df.columns = df.columns.str.strip().str.lower()

                    # 确保日期列为datetime类型
                    date_columns = ['日期', 'date', '时间', 'time']
                    for col in date_columns:
                        if col in df.columns:
                            df[col] = pd.to_datetime(df[col])
                            df.set_index(col, inplace=True)
                            break

                    # 保存到缓存
                    self._save_to_cache(df, cache_key)

                    # 避免请求过快
                    time.sleep(self.request_delay)

                    return df

            except Exception as e:
                print(f"获取{symbol}数据失败 (尝试 {attempt+1}/{self.retry_times}): {e}")
                time.sleep(1)  # 失败后等待1秒再重试

        print(f"无法获取{symbol}数据，已重试{self.retry_times}次")
        return None

    def get_stock_hist_data(self,
                           symbol_list: Union[str, List[str]],
                           period: str = "daily",
                           start_date: str = None,
                           end_date: str = None,
                           adjust: str = "qfq",
                           use_cache: bool = True,
                           parallel: bool = True) -> Dict[str, pd.DataFrame]:
        """
        获取股票历史数据（改进版）

        Args:
            symbol_list: 股票代码或代码列表
            period: 周期，支持'daily'（日线）等
            start_date: 开始日期，格式'YYYYMMDD'
            end_date: 结束日期，格式'YYYYMMDD'
            adjust: 复权类型，'qfq'（前复权），'hfq'（后复权），''（不复权）
            use_cache: 是否使用缓存
            parallel: 是否并行下载

        Returns:
            字典，键为股票代码，值为数据框
        """
        # 参数处理
        if isinstance(symbol_list, str):
            symbol_list = [symbol_list]

        if end_date is None:
            end_date = datetime.now().strftime('%Y%m%d')

        if start_date is None:
            # 默认获取最近一年的数据
            start_date = (datetime.now() - timedelta(days=365)).strftime('%Y%m%d')

        # 准备参数
        kwargs = {
            'data_type': 'daily',
            'period': period,
            'start_date': start_date,
            'end_date': end_date,
            'adjust': adjust
        }

        result_dict = {}

        # 并行下载
        if parallel and len(symbol_list) > 1:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                # 提交任务
                future_to_symbol = {
                    executor.submit(self._fetch_single_stock, symbol, **kwargs): symbol
                    for symbol in symbol_list
                }

                # 处理结果
                for future in tqdm(concurrent.futures.as_completed(future_to_symbol),
                                 total=len(symbol_list), desc="下载数据"):
                    symbol = future_to_symbol[future]
                    try:
                        df = future.result()
                        if df is not None:
                            result_dict[symbol] = df
                    except Exception as e:
                        print(f"处理{symbol}时出错: {e}")

        # 串行下载
        else:
            for symbol in tqdm(symbol_list, desc="下载数据"):
                df = self._fetch_single_stock(symbol, **kwargs)
                if df is not None:
                    result_dict[symbol] = df

        return result_dict

    def get_stock_hist_data_combined(self,
                                    symbol_list: Union[str, List[str]],
                                    **kwargs) -> pd.DataFrame:
        """
        获取股票历史数据并合并为单一数据框

        Args:
            symbol_list: 股票代码或代码列表
            **kwargs: 传递给get_stock_hist_data的参数

        Returns:
            合并后的数据框，包含多级索引（日期，股票代码）
        """
        data_dict = self.get_stock_hist_data(symbol_list, **kwargs)

        if not data_dict:
            return pd.DataFrame()

        # 合并数据
        combined_data = []
        for symbol, df in data_dict.items():
            if not df.empty:
                # 添加股票代码列（如果还没有）
                if 'symbol' not in df.columns:
                    df['symbol'] = symbol

                # 重置索引以便合并
                df_reset = df.reset_index()

                # 确保有日期列
                if '日期' in df_reset.columns:
                    df_reset.rename(columns={'日期': 'date'}, inplace=True)

                combined_data.append(df_reset)

        if combined_data:
            combined_df = pd.concat(combined_data, ignore_index=True)

            # 设置多级索引
            if 'date' in combined_df.columns and 'symbol' in combined_df.columns:
                combined_df['date'] = pd.to_datetime(combined_df['date'])
                combined_df.set_index(['date', 'symbol'], inplace=True)
                combined_df.sort_index(inplace=True)

            return combined_df
        else:
            return pd.DataFrame()

    def get_index_components(self, index_code: str = "000300") -> List[str]:
        """
        获取指数成分股

        Args:
            index_code: 指数代码
                '000300' - 沪深300
                '000905' - 中证500
                '000016' - 上证50

        Returns:
            成分股代码列表
        """
        try:
            if index_code == "000300":
                df = ak.index_stock_cons_csindex(symbol="000300")
            elif index_code == "000905":
                df = ak.index_stock_cons_csindex(symbol="000905")
            elif index_code == "000016":
                df = ak.stock_sz_50_spot()
            else:
                print(f"暂不支持的指数代码: {index_code}")
                return []

            if df is not None and not df.empty:
                # 提取股票代码，不同接口返回的列名可能不同
                for col in ['成分券代码', '品种代码', 'code']:
                    if col in df.columns:
                        return df[col].astype(str).str.zfill(6).tolist()

        except Exception as e:
            print(f"获取指数成分股失败: {e}")

        return []

    def get_basic_info(self, symbol: str) -> Dict:
        """
        获取股票基本信息

        Args:
            symbol: 股票代码

        Returns:
            股票基本信息字典
        """
        try:
            info = ak.stock_individual_info_em(symbol=symbol)
            if info is not None and not info.empty:
                return info.set_index('item')['value'].to_dict()
        except Exception as e:
            print(f"获取{symbol}基本信息失败: {e}")

        return {}

    def validate_data(self, df: pd.DataFrame, symbol: str) -> bool:
        """
        验证数据质量

        Args:
            df: 数据框
            symbol: 股票代码

        Returns:
            数据是否有效
        """
        if df is None or df.empty:
            print(f"{symbol}: 数据为空")
            return False

        # 检查必要的列
        required_columns = ['开盘', '收盘', '最高', '最低', '成交量']
        missing_cols = [col for col in required_columns if col not in df.columns]

        if missing_cols:
            print(f"{symbol}: 缺少必要列 {missing_cols}")
            return False

        # 检查数据完整性
        if len(df) < 10:
            print(f"{symbol}: 数据量太少 ({len(df)} 条)")
            return False

        # 检查异常值
        if (df['收盘'] <= 0).any():
            print(f"{symbol}: 存在非正收盘价")
            return False

        return True


In [25]:
if __name__ == '__main__':
    #初始化
    as_share = ASshare(cache_dir='./stock_data', max_workers=10)

    # ex.1:
    print("获取单支股票数据...")
    single_data = as_share.get_stock_hist_data(
        symbol_list='000001',
        period='daily',
        start_date='20210101',
        end_date='20210131',
        adjust='qfq',
    )
    if single_data:
        print(f"获取到{len(single_data)}只股票数据")
        for symbol, df in single_data.items():
            print(f"股票{symbol}数据量：{len(df)}")
            print(f"股票{symbol}数据形状：{df.shape}")
            print(df.head())

获取单支股票数据...


下载数据: 100%|██████████| 1/1 [00:00<00:00,  4.16it/s]

获取到1只股票数据
股票000001数据量：20
股票000001数据形状：(20, 12)
              股票代码     开盘     收盘     最高     最低      成交量           成交额    振幅  \
日期                                                                            
2021-01-04  000001  16.84  16.34  16.84  16.18  1554216  2.891682e+09  3.86   
2021-01-05  000001  16.14  15.91  16.22  15.54  1821352  3.284607e+09  4.16   
2021-01-06  000001  15.82  17.30  17.30  15.74  1934945  3.648522e+09  9.81   
2021-01-07  000001  17.26  17.64  17.72  16.97  1584185  3.111275e+09  4.34   
2021-01-08  000001  17.64  17.59  17.84  17.05  1195473  2.348316e+09  4.48   

             涨跌幅   涨跌额   换手率  symbol  
日期                                    
2021-01-04 -4.33 -0.74  0.80  000001  
2021-01-05 -2.63 -0.43  0.94  000001  
2021-01-06  8.74  1.39  1.00  000001  
2021-01-07  1.97  0.34  0.82  000001  
2021-01-08 -0.28 -0.05  0.62  000001  


In [30]:
# ex.2 获取多汁股票
symbols = ['000001','000002','600519']

combined_df = as_share.get_stock_hist_data_combined(
    symbol_list=symbols,
    period='daily',
    start_date='20210101',
    end_date='20210131',
    adjust='qfq',
    parallel=True
)

if not combined_df.empty:
    print(f"合并后形状:{combined_df.shape}")
    print("\n查看贵州茅台数据:")
    print(combined_df.xs('600519', level='symbol').head())

    combined_df.to_csv('combined_data.csv')


下载数据: 100%|██████████| 3/3 [00:00<00:00, 11.14it/s]

合并后形状:(60, 11)

查看贵州茅台数据:
              股票代码       开盘       收盘       最高       最低    成交量           成交额  \
date                                                                          
2021-01-04  600519  1785.77  1782.79  1790.78  1769.60  43514  8.686920e+09   
2021-01-05  600519  1775.79  1845.24  1845.24  1768.25  52116  1.059256e+10   
2021-01-06  600519  1850.59  1885.79  1910.79  1821.82  47410  9.882594e+09   
2021-01-07  600519  1882.79  1925.79  1925.79  1860.79  37931  7.991179e+09   
2021-01-08  600519  1927.79  1875.79  1936.67  1848.81  53862  1.134520e+10   

              振幅   涨跌幅    涨跌额   换手率  
date                                 
2021-01-04  1.19 -0.06  -1.00  0.35  
2021-01-05  4.32  3.50  62.45  0.41  
2021-01-06  4.82  2.20  40.55  0.38  
2021-01-07  3.45  2.12  40.00  0.30  
2021-01-08  4.56 -2.60 -50.00  0.43  


In [31]:
combined_df

股票代码       开盘       收盘       最高       最低      成交量  \
date       symbol                                                        
2021-01-04 000001  000001    16.84    16.34    16.84    16.18  1554216   
           000002  000002    25.49    24.87    25.49    24.61  1468441   
           600519  600519  1785.77  1782.79  1790.78  1769.60    43514   
2021-01-05 000001  000001    16.14    15.91    16.22    15.54  1821352   
           000002  000002    24.87    25.00    25.02    24.19  1162658   
           600519  600519  1775.79  1845.24  1845.24  1768.25    52116   
2021-01-06 000001  000001    15.82    17.30    17.30    15.74  1934945   
           000002  000002    24.89    25.84    25.84    24.82  1048801   
           600519  600519  1850.59  1885.79  1910.79  1821.82    47410   
2021-01-07 000001  000001    17.26    17.64    17.72    16.97  1584185   
           000002  000002    26.09    25.88    26.59    25.48  1226756   
           600519  600519  1882.79  1925.79  1925.79  1860.79    37931   
2021-01-08 000001  000001    17.64    17.59    17.84    17.05  1195473   
           000002  000002    26.07    26.43    26.54    25.90  1028563   
           600519  600519  1927.79  1875.79  1936.67  1848.81    53862   
2021-01-11 000001  000001    17.74    18.12    18.38    17.74  1790457   
           000002  000002    26.59    26.87    27.44    26.36  1388121   
           600519  600519  1875.79  1885.52  1936.29  1844.39    51080   
2021-01-12 000001  000001    18.13    18.74    18.74    17.92  1288166   
           000002  000002    26.87    26.79    26.89    26.46   980654   
           600519  600519  1873.79  1946.69  1946.69  1870.79    41059   
2021-01-13 000001  000001    18.74    18.44    18.75    18.14  1049120   
           000002  000002    26.99    26.99    27.31    26.69   940504   
           600519  600519  1949.79  1949.79  1959.12  1920.79    34390   
2021-01-14 000001  000001    18.42    17.91    18.63    17.69  1130156   
           000002  000002    27.02    27.08    27.88    26.89  1216323   
           600519  600519  1941.79  1919.79  1948.79  1901.90    36152   
2021-01-15 000001  000001    18.74    18.74    19.69    18.56  2480689   
           000002  000002    27.10    27.04    27.82    26.97  1123372   
           600519  600519  1900.79  1867.79  1920.14  1814.79    59509   
2021-01-18 000001  000001    18.94    20.44    20.52    18.94  2157302   
           000002  000002    27.29    28.35    28.35    26.92  1258282   
           600519  600519  1846.85  1848.79  1877.27  1826.06    39144   
2021-01-19 000001  000001    20.25    20.08    20.58    19.79  1242088   
           000002  000002    28.28    28.37    29.44    27.81  1631688   
           600519  600519  1858.90  1795.20  1882.09  1792.64    46345   
2021-01-20 000001  000001    19.89    20.21    20.71    19.86  1280793   
           000002  000002    28.16    27.59    28.29    27.44   961435   
           600519  600519  1793.79  1826.42  1841.28  1768.29    45082   
2021-01-21 000001  000001    20.24    19.97    20.54    19.89   944779   
           000002  000002    27.39    27.19    27.79    27.11   875253   
           600519  600519  1833.79  1855.79  1875.81  1826.79    38310   
2021-01-22 000001  000001    19.97    19.77    19.97    19.25   924434   
           000002  000002    27.04    26.35    27.04    26.10  1058091   
           600519  600519  1854.79  1865.74  1885.79  1845.89    29248   
2021-01-25 000001  000001    19.46    20.23    20.34    19.17  1028365   
           000002  000002    26.35    26.45    26.69    25.65   973750   
           600519  600519  1868.79  1960.79  1965.29  1868.79    55957   
2021-01-26 000001  000001    20.04    20.11    21.06    20.04  1126721   
           000002  000002    26.35    26.79    27.24    26.24   816655   
           600519  600519  1970.79  1927.68  1972.79  1910.90    31747   
2021-01-27 000001  000001    20.05    20.82    21.21    20.05  1294153   
           000002  000002